<a href="https://colab.research.google.com/github/priyanshgupta1998/All_codes/blob/master/Image_captioning_RNN%2BCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/yashk2810/Image-Captioning/blob/master/Image%20Captioning%20InceptionV3.ipynb

#RNN  

https://www.geeksforgeeks.org/introduction-to-recurrent-neural-network/

LSTM can by default retain the information for long period of time.

#LSTM
https://www.geeksforgeeks.org/deep-learning-introduction-to-long-short-term-memory/  

LSTM has a chain structure that contains four neural networks and different memory blocks called cells.

In [1]:
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from tqdm import tqdm
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Concatenate, Activation, Flatten ,Add
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
import nltk

Using TensorFlow backend.


In [0]:
import zipfile
zip_ref = zipfile.ZipFile('/home/Flickr8k_text.zip', 'r')
zip_ref.extractall('/home')
zip_ref.close()

In [0]:
token = '/home/Flickr8k.token.txt'

In [0]:
captions = open(token, 'r').read().strip().split('\n')

#Creating a dictionary containing all the captions of the images

In [14]:
l = []
for i, row in enumerate(captions):     # 'i' is from 0 to 40459
    if i<10:
        l.append(row)
l

['1000268201_693b08cb0e.jpg#0\tA child in a pink dress is climbing up a set of stairs in an entry way .',
 '1000268201_693b08cb0e.jpg#1\tA girl going into a wooden building .',
 '1000268201_693b08cb0e.jpg#2\tA little girl climbing into a wooden playhouse .',
 '1000268201_693b08cb0e.jpg#3\tA little girl climbing the stairs to her playhouse .',
 '1000268201_693b08cb0e.jpg#4\tA little girl in a pink dress going into a wooden cabin .',
 '1001773457_577c3a7d70.jpg#0\tA black dog and a spotted dog are fighting',
 '1001773457_577c3a7d70.jpg#1\tA black dog and a tri-colored dog playing with each other on the road .',
 '1001773457_577c3a7d70.jpg#2\tA black dog and a white dog with brown spots are staring at each other in the street .',
 '1001773457_577c3a7d70.jpg#3\tTwo dogs of different breeds looking at each other on the road .',
 '1001773457_577c3a7d70.jpg#4\tTwo dogs on pavement moving toward each other .']

In [17]:
l = []
for i, row in enumerate(captions):     # 'i' is from 0 to 40459
    if i<10:
        row = row.split('\t')
        row[0] = row[0][:len(row[0])-2]
        l.append(row[0])
l

['1000268201_693b08cb0e.jpg',
 '1000268201_693b08cb0e.jpg',
 '1000268201_693b08cb0e.jpg',
 '1000268201_693b08cb0e.jpg',
 '1000268201_693b08cb0e.jpg',
 '1001773457_577c3a7d70.jpg',
 '1001773457_577c3a7d70.jpg',
 '1001773457_577c3a7d70.jpg',
 '1001773457_577c3a7d70.jpg',
 '1001773457_577c3a7d70.jpg']

In [22]:
l = []
for i, row in enumerate(captions):     # 'i' is from 0 to 40459
    row = row.split('\t')
    row[0] = row[0][:len(row[0])-2]
    l.append(row[0])
print(len(l))
print(len(np.unique(l)))     # unique images

40460
8092


In [31]:
d = {}
for i, row in enumerate(captions):
    row = row.split('\t')
    row[0] = row[0][:len(row[0])-2]
    if row[0] in d:
        d[row[0]].append(row[1])
    else:
        d[row[0]] = [row[1]]

#d
        
k = {}
for i , data in enumerate(d.items()):
    if i<10:
        k[i] = data
k
        

{0: ('1000268201_693b08cb0e.jpg',
  ['A child in a pink dress is climbing up a set of stairs in an entry way .',
   'A girl going into a wooden building .',
   'A little girl climbing into a wooden playhouse .',
   'A little girl climbing the stairs to her playhouse .',
   'A little girl in a pink dress going into a wooden cabin .']),
 1: ('1001773457_577c3a7d70.jpg',
  ['A black dog and a spotted dog are fighting',
   'A black dog and a tri-colored dog playing with each other on the road .',
   'A black dog and a white dog with brown spots are staring at each other in the street .',
   'Two dogs of different breeds looking at each other on the road .',
   'Two dogs on pavement moving toward each other .']),
 2: ('1002674143_1b742ab4b8.jpg',
  ['A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .',
   'A little girl is sitting in front of a large painted rainbow .',
   'A small girl in the grass plays with fingerpaints in front of a white canvas

In [34]:
d['1000268201_693b08cb0e.jpg']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

In [0]:
images = 'Flickr8k_Dataset/Flicker8k_Dataset/'

Python Glob() Function To Match Path, Directory, File Names with Examples. The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order.  but * , ? , and character ranges expressed with [] will be correctly matched.

In [0]:
# Contains all the images
img = glob.glob(images+'*.jpg')

In [37]:
img[:5]

[]